In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon_Kitchen").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")
# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
# Drop null and duplicates
dropna_df = df.dropna()
cleaned_df = dropna_df.dropDuplicates()

In [6]:
# Count the number of records (rows) in the dataset.
print(cleaned_df.count())

4879961


 **Examine the schema** 

In [7]:
# Transform the dataset to fit the tables in the schema file.
# Be sure the DataFrames match in data type and in column name.
cleaned_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# Create customers table
from pyspark.sql.functions import desc
customer_count_df = cleaned_df.select("customer_id").groupby("customer_id")\
  .agg({"customer_id":"count"})
customer_count_df = customer_count_df.withColumnRenamed("count(customer_id)", "customer_count")
customer_count_df.show(truncate=False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|16308995   |2             |
|4709446    |1             |
|31546122   |2             |
|25866981   |7             |
|11537699   |4             |
|50481713   |2             |
|14863908   |2             |
|52964170   |1             |
|51863594   |1             |
|42176175   |5             |
|39258095   |1             |
|52456816   |5             |
|23141101   |3             |
|40302623   |2             |
|18614361   |1             |
|25748567   |2             |
|15185969   |9             |
|30414919   |8             |
|37589671   |1             |
|17018430   |1             |
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create user dataframe to match review_id_table table
review_id_table = cleaned_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002B10PD9ZFM|   24197845|B000LTOCSG|     630072149| 2009-07-20|
|R1007DQP1BG5WJ|   17990743|B007M2OHEY|     362955205| 2014-09-12|
|R100AIJ0ZFN8CZ|   33816354|B0001ZYYQU|     331822491| 2007-03-09|
|R100C48B994FWD|   13231777|B00IK59T1S|     744019201| 2014-11-26|
|R100E300CKVFYD|   53006884|B00833DQCU|     298219798| 2013-12-08|
|R100GBKBV767AA|   14608024|B0000CDVD2|     184834728| 2014-04-30|
|R100I1XICZFNHD|   46627762|B0014NKJTO|      21858491| 2014-03-04|
|R100KXBN0DJ74A|   42223395|B000P9GWFS|     826643513| 2012-07-04|
|R100VC4OCYFFWM|   29497970|B0016KXAD2|     641397529| 2015-08-26|
|R100YFR4KUB3GC|   13368891|B003LYVXJ4|     510828298| 2013-12-08|
|R1011FF2WI90BO|   38569985|B000A7W4YS|     719164168| 2005-12-15|
|R101KSJLZDFCG2|   19893443|B0085MQPSG|     839082593| 2014-07

In [10]:
# Create user dataframe to match products table 
products = cleaned_df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000LTOCSG|Replacement Filte...|
|B007M2OHEY|Fred & Friends E....|
|B0001ZYYQU|KitchenAid Artisa...|
|B00IK59T1S|PUR 7 Cup Pitcher...|
|B00833DQCU|Prepworks from Pr...|
|B0000CDVD2|Oxo Good Grips Co...|
|B0014NKJTO|9 Inch Vinyl Sede...|
|B000P9GWFS|KitchenAid Profes...|
|B0016KXAD2|          Party Hats|
|B003LYVXJ4|Aluminum Wire Sof...|
|B000A7W4YS|Braun Tassimo TA ...|
|B0085MQPSG|BigMouth Inc The ...|
|B000H8E03I|Graviti Electroni...|
|B003F6GKZU|Chef'n PepperBall...|
|B000KESQ1G|Ateco Professiona...|
|B00004S7V7|Microplane 40001 ...|
|B004QGXNT4|Kitchen Elements ...|
|B0007SXIMM|KitchenAid KFP740...|
|B0006MEI5A|Oxi Clean Stain R...|
|B0000DE7JR|Stainless Steel R...|
+----------+--------------------+
only showing top 20 rows



Postgres Setup

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ucsd-fyc.ckcik1qdrfa0.us-east-2.rds.amazonaws.com:5432/amazon_review"
config = {"user":"root", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [16]:
# Write DataFrame to active_user table in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [17]:
# Write dataframe to products table in RDS

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [18]:
# Write dataframe to customers table in RDS

customer_count_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored